In [ ]:
import matplotlib.pyplot as plt
import cv2  
import scipy.ndimage.filters as filters
import numpy as np
import pandas as pd
import os
from pm import plot_matches

fontanna1 = cv2.imread('fontanna1.jpg')
fontanna2 = cv2.imread('fontanna2.jpg')

# Implementacja metody Harrisa 6.1

In [ ]:
def h_value(grayscale_image, sobel_size, gauss_size):
    sobel_x = cv2.Sobel(grayscale_image, cv2.CV_32F, 1, 0, ksize=sobel_size)
    sobel_y = cv2.Sobel(grayscale_image, cv2.CV_32F, 0, 1, ksize=sobel_size)
    
    # iloczyny pochodnych kierunkowych
    Ixx = sobel_x * sobel_x
    Iyy = sobel_y * sobel_y
    Ixy = sobel_x * sobel_y
    
    # rozmycie filtrem Gaussa
    Ixx = cv2.GaussianBlur(Ixx, (gauss_size, gauss_size), 0)
    Iyy = cv2.GaussianBlur(Iyy, (gauss_size, gauss_size), 0)
    Ixy = cv2.GaussianBlur(Ixy, (gauss_size, gauss_size), 0)

    # obliczenie wartości H
    K = 0.05
    h = (Ixx * Iyy - Ixy * Ixy) - K * (Ixx + Iyy) ** 2
    h = cv2.normalize(h, None, 0, 1, cv2.NORM_MINMAX, cv2.CV_32F)
    return h

def find_max ( image , size , threshold ) : # size - maximum filter mask size
    data_max = filters . maximum_filter ( image , size )
    maxima = ( image == data_max )
    diff = image > threshold
    maxima [ diff == 0] = 0
    return np.nonzero( maxima )

def draw_stars(image, coordinates):
    plt.figure()
    plt.imshow(image)
    for coord in zip(*coordinates):
        plt.plot(
            coord[1], coord[0], "*", color="r"
        )
    plt.axis("off")
    plt.show()

In [ ]:
threshold = 0.55

fontanna1_gray = cv2.cvtColor(fontanna1, cv2.COLOR_BGR2GRAY)
fontanna2_gray = cv2.cvtColor(fontanna2, cv2.COLOR_BGR2GRAY)

h1 = h_value(fontanna1_gray, 7, 7)
h2 = h_value(fontanna2_gray, 7, 7)

corners1 = find_max(h1, 7, threshold)
corners2 = find_max(h2, 7, threshold)

draw_stars(fontanna1, corners1)
draw_stars(fontanna2, corners2)

In [ ]:
budynek1 = cv2.imread('budynek1.jpg')
budynek2 = cv2.imread('budynek2.jpg')

budynek1_gray = cv2.cvtColor(budynek1, cv2.COLOR_BGR2GRAY)
budynek2_gray = cv2.cvtColor(budynek2, cv2.COLOR_BGR2GRAY)

h3 = h_value(budynek1_gray, 7, 7)
h4 = h_value(budynek2_gray, 7, 7)

corners3 = find_max(h3, 7, threshold)
corners4 = find_max(h4, 7, threshold)

draw_stars(budynek1, corners3)
draw_stars(budynek2, corners4)

# Deskrypcja 6.2

In [ ]:
def describe_region(image, coordinates, size):
    pts = list(filter(lambda x: x[0] >= size and x[0] < image.shape[0] - size 
                      and x[1] >= size and x[1] < image.shape[1] - size, zip(coordinates[0], coordinates[1])))
    patches = []
    for pt in pts:
        patch = image[pt[0]-size:pt[0]+size+1, pt[1]-size:pt[1]+size+1]
        patches.append(patch.flatten())
    return list(zip(patches, pts))

def compare_patches(desc1, desc2, n):
    comparison = []

    for v1, pts1 in desc1:
        min_val = float('inf')
        best_val = None
        for v2, pts2 in desc2:
            diff = sum(abs(v1 - v2))
            if diff < min_val:
                min_val = diff
                best_val = [pts1, pts2, diff]

        comparison.append(best_val)

    comparison.sort(key=lambda x: x[2])
    comparison = comparison[:n]

    return comparison

### fontanna1, fontanna2, budynek1, budynek2

In [ ]:
patch_size = 15
n = 20

def f1f2b1b262():
    patches1 = describe_region(fontanna1_gray, corners1, patch_size)
    patches2 = describe_region(fontanna2_gray, corners2, patch_size)
    similar_patches = compare_patches(patches1, patches2, n)
    plot_matches(fontanna1_gray, fontanna2_gray, similar_patches)

    patches3 = describe_region(budynek1_gray, corners3, patch_size)
    patches4 = describe_region(budynek2_gray, corners4, patch_size)

    similar_patches2 = compare_patches(patches3, patches4, n)
    plot_matches(budynek1_gray, budynek2_gray, similar_patches2)

f1f2b1b262()

### fontanna_pow, fontanna1

In [ ]:
def fpowf162():
    fontanna_pow = cv2.imread('fontanna_pow.jpg')
    fontanna_pow_gray = cv2.cvtColor(fontanna_pow, cv2.COLOR_BGR2GRAY)
    h5 = h_value(fontanna_pow_gray, 7, 7)
    corners5 = find_max(h5, 7, threshold)

    patches5 = describe_region(fontanna_pow_gray, corners5, patch_size)
    patches6 = describe_region(fontanna1_gray, corners1, patch_size)

    similar_patches3 = compare_patches(patches5, patches6, n)
    plot_matches(fontanna_pow_gray, fontanna1_gray, similar_patches3)

fpowf162()

### eiffel1, eiffel2

In [ ]:
def e1e262():
    eiffel1 = cv2.imread('eiffel1.jpg')
    eiffel2 = cv2.imread('eiffel2.jpg')

    eiffel1_gray = cv2.cvtColor(eiffel1, cv2.COLOR_BGR2GRAY)
    eiffel2_gray = cv2.cvtColor(eiffel2, cv2.COLOR_BGR2GRAY)

    h6 = h_value(eiffel1_gray, 7, 7)
    h7 = h_value(eiffel2_gray, 7, 7)

    corners6 = find_max(h6, 7, threshold)
    corners7 = find_max(h7, 7, threshold)

    patches7 = describe_region(eiffel1_gray, corners6, patch_size)
    patches8 = describe_region(eiffel2_gray, corners7, patch_size)

    similar_patches4 = compare_patches(patches7, patches8, n)
    plot_matches(eiffel1_gray, eiffel2_gray, similar_patches4)

e1e262()

# afiniczne zmiany jasności

In [ ]:
def describe_region(image, coordinates, size):
    pts = list(filter(lambda x: x[0] >= size and x[0] < image.shape[0] - size 
                      and x[1] >= size and x[1] < image.shape[1] - size, zip(coordinates[0], coordinates[1])))
    patches = []
    for pt in pts:
        patch = image[pt[0]-size:pt[0]+size+1, pt[1]-size:pt[1]+size+1]
        patch = patch.flatten()
        mean = np.mean(patch)
        std = np.std(patch)
        patch = (patch - mean) / std
        patches.append(patch)
    return list(zip(patches, pts))

In [ ]:
f1f2b1b262()
fpowf162()
e1e262()

# 6.3 Implementacja elementów algorytmu ORB 

In [ ]:
def orb_matching(image_path1, image_path2, n_features=500):
    def detect_fast_keypoints(image):
        fast = cv2.FastFeatureDetector_create(threshold=25, nonmaxSuppression=True)
        keypoints = fast.detect(image, None)
        return keypoints

    def compute_harris_measure(image, keypoints):
        harris_response = cv2.cornerHarris(image, blockSize=2, ksize=3, k=0.04)
        for kp in keypoints:
            x, y = int(kp.pt[0]), int(kp.pt[1])
            kp.response = harris_response[y, x]
        return keypoints

    def non_maximum_suppression(keypoints, radius=3):
        if not keypoints:
            return []
        
        keypoints = sorted(keypoints, key=lambda x: x.response, reverse=True)
        suppressed = []
        for i, kp in enumerate(keypoints):
            keep = True
            for j in range(len(suppressed)):
                if cv2.norm(kp.pt, suppressed[j].pt) < radius:
                    keep = False
                    break
            if keep:
                suppressed.append(kp)
        return suppressed

    def filter_keypoints_by_patch(keypoints, image_shape, patch_size=31):
        margin = patch_size // 2
        return [
            kp for kp in keypoints
            if margin <= kp.pt[0] < image_shape[1] - margin and margin <= kp.pt[1] < image_shape[0] - margin
        ]

    def compute_orb_descriptors(image, keypoints):
        orb = cv2.ORB_create(nfeatures=n_features)
        keypoints, descriptors = orb.compute(image, keypoints)
        return keypoints, descriptors

    # Load images
    image1 = cv2.imread(image_path1, cv2.IMREAD_GRAYSCALE)
    image2 = cv2.imread(image_path2, cv2.IMREAD_GRAYSCALE)

    if image1 is None or image2 is None:
        print("Error loading images.")
        return

    # Process image 1
    keypoints1 = detect_fast_keypoints(image1)
    keypoints1 = compute_harris_measure(image1, keypoints1)
    keypoints1 = non_maximum_suppression(keypoints1)
    keypoints1 = filter_keypoints_by_patch(keypoints1, image1.shape, patch_size=31)
    keypoints1, descriptors1 = compute_orb_descriptors(image1, keypoints1)

    # Process image 2
    keypoints2 = detect_fast_keypoints(image2)
    keypoints2 = compute_harris_measure(image2, keypoints2)
    keypoints2 = non_maximum_suppression(keypoints2)
    keypoints2 = filter_keypoints_by_patch(keypoints2, image2.shape, patch_size=31)
    keypoints2, descriptors2 = compute_orb_descriptors(image2, keypoints2)

    # Match descriptors
    matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = matcher.match(descriptors1, descriptors2)
    matches = sorted(matches, key=lambda x: x.distance)

    # Visualize results
    result = cv2.drawMatches(image1, keypoints1, image2, keypoints2, matches[:50], None, flags=2)
    plt.figure(figsize=(12, 6))
    plt.imshow(result)
    plt.axis('off')
    plt.show()


In [ ]:
orb_matching('fontanna1.jpg', 'fontanna2.jpg')

In [ ]:
orb_matching('budynek1.jpg', 'budynek2.jpg')

In [ ]:
orb_matching('eiffel1.jpg', 'eiffel2.jpg')

In [ ]:
orb_matching('eiffel_rot.jpg', 'eiffel2.jpg')

# 6.4

In [ ]:
left_panorama = cv2.imread('left_panorama.jpg')
right_panorama = cv2.imread('right_panorama.jpg')

left_panorama_gray = cv2.cvtColor(left_panorama, cv2.COLOR_BGR2GRAY)
right_panorama_gray = cv2.cvtColor(right_panorama, cv2.COLOR_BGR2GRAY)

ORB = cv2.ORB_create()

keypoints1, descriptors1 = ORB.detectAndCompute(left_panorama_gray, None)
keypoints2, descriptors2 = ORB.detectAndCompute(right_panorama_gray, None)

kp_img_left = cv2.drawKeypoints(left_panorama_gray, keypoints1, None, color=(0, 0, 255))
kp_img_right = cv2.drawKeypoints(right_panorama_gray, keypoints2, None, color=(0, 0, 255))

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(kp_img_left)
plt.title('Keypoints in Left Panorama')
plt.axis('off')
plt.subplot(1, 2, 2)
plt.imshow(kp_img_right)
plt.title('Keypoints in Right Panorama')
plt.axis('off')


In [ ]:
bf = cv2.BFMatcher(cv2.NORM_HAMMING)

matches = bf.knnMatch(descriptors1, descriptors2, k=2)
good_matches = []
for m, n in matches:
    if m.distance < 0.5* n.distance:
        good_matches.append([m])

img_matches = cv2.drawMatchesKnn(left_panorama_gray, keypoints1, right_panorama_gray, keypoints2, good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
plt.figure(figsize=(10, 5))
plt.imshow(img_matches)
plt.title('Matches between Panoramas')
plt.axis('off')
plt.show()

In [ ]:
keypointsL = np.float32([kp.pt for kp in keypoints1])
keypointsR = np.float32([kp.pt for kp in keypoints2])

ptsA = np.float32([keypointsL[m[0].queryIdx] for m in good_matches])
ptsB = np.float32([keypointsR[m[0].trainIdx] for m in good_matches])

homography_matrix, status = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, 5.0)

result = cv2.warpPerspective(left_panorama, homography_matrix, (left_panorama.shape[1] + right_panorama.shape[1], right_panorama.shape[0]))

result[0:right_panorama.shape[0], 0:right_panorama.shape[1]] = right_panorama

result = result[:, :1280]

In [ ]:
plt.subplots(1, 3, figsize=(15, 10))
plt.subplot(131)
plt.imshow(left_panorama)
plt.title('Left panorama')
plt.subplot(132)
plt.imshow(right_panorama)
plt.title('Right panorama')
plt.subplot(133)
plt.imshow(result)
plt.title('Stitched panorama')
plt.show()